# Xmas Bayesian Optimisation Guessing Game
This notebook implements a xmas-themed guessing game to try and find the maximum of a black-box function.

In [129]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd
import altair as alt
import gpflow
import blackbox as bb
from importlib import reload
reload(bb);

## The scenario
It is Xmas Eve and Santa has to deliver his presents to all the kids around the world.
Unfortunately it has been a really cold winter and some houses are completely blanketed by snow.

Santa does have a shovel and could dig down to the chimney of each house to deliver his presents if only he knew where the chimneys were.

Fortunately Rudolph can help.
He can measure the depth of the snow at any point with his antlers but unfortunately this takes some time and the measurements are noisy.

Santa only has one night to deliver his presents, how can he find the chimneys in time?

### Skyline
Let's see what a typical house's skyline looks like.
`x` represents the location and `depth` measures the depth below the snowline.

In [130]:
x, depth = bb.random_submerged_skyline(100)
house = pd.DataFrame(dict(x=x, depth=depth))
house_chart = (
    alt.Chart(house)
    .mark_line()
    .encode(x='x', y='depth'))
house_chart.properties(width=800, height=300)

alt.Chart(...)

### Grid-search strategy
One strategy to find the chimney is to lay down a grid and ask Rudolph to measure the height at each grid point.

Let's fix the standard deviation of the Rudolph's measurement error and choose a number of grid points:

In [131]:
error_sd = widgets.FloatSlider(
    value=.04,
    min=0.02,
    max=.1,
    step=0.01,
    description='error sd:',
    readout_format='.2f',
)
npoints = widgets.IntSlider(
    value=20,
    min=3,
    max=100,
    step=1,
    description='grid size:',
    readout_format='d'
)
display(error_sd)
display(npoints)

FloatSlider(value=0.04, description='error sd:', max=0.1, min=0.02, step=0.01)

IntSlider(value=20, description='grid size:', min=3)

Rudolph makes his measurements.

In [132]:
measurements = pd.DataFrame(dict(x=np.linspace(house['x'].min(), house['x'].max(), npoints.value)))
f = interp1d(house['x'], house['depth'])
measurements['f'] = f(measurements['x'])
measurements['y'] = measurements['f'] + bb.rng.normal(0, error_sd.value, size=npoints.value)
measurements_chart = (
    alt.Chart(measurements)
    .mark_point(color='red')
    .encode(x='x', y='y'))
(measurements_chart + house_chart).properties(width=800, height=300)

alt.LayerChart(...)

We can see that there are many wasted measurements in locations where we are reasonably sure the maximum is not located.
In addition it is still difficult to be sure where the chimney is.

How can we do better?

## Bayesian optimisation
This is where Bayesian optimisation comes in.

Bayesian optimisation (BO) requires:
- a prior on the underlying function to be optimised (some understanding of how houses are buried)
- a prior on the measurement error (some understanding of how good Rudolph is at measuring depth)

Using some underlying theory, Bayesian optimisation then iterates around the following loop:
- choose the next measurement location so as to minimise the total number of measurements required (the acquisition function)
- ask the black-box function for a noisy measurement at that location
- update its model of the underlying function and measurement error (computes a posterior given prior and data)

Some stopping criterion is used to avoid iterating indefinitely.
Bayesian optimisation returns the posterior for the underlying function to use as you see fit.
In particular you may wish to obtain a posterior over the location of the maximum of the function.

## Optimisation guessing game
However, before we try Bayesian optimisation, we will try to optimise a black-box function ourselves.
Your task will be to find the maximum of the underlying function using as few function evaluations as possible.

Unfortunately we do not have a suitable Bayesian model for house skylines so we will use a Gaussian process prior on functions instead.
Some typical functions are shown below to give you an idea how the prior behaves.

In [133]:
blackbox1 = bb.GPBlackBox(ndim=1)
xx = blackbox1.xgrid(npoints=100)[:, 0]
prior_samples = pd.concat([
    pd.DataFrame(dict(x=xx, f=bb.prior_draw(blackbox1.kernel, xx), sample=s))
    for s in range(3)])
prior_chart = (
    alt.Chart(prior_samples)
    .mark_line()
    .encode(x='x', y='f', color='sample:N'))
prior_chart.properties(width=800, height=300)

alt.Chart(...)

### A function of one input
Now it is your turn.
You have to optimise a function of one input with domain `[-1, 1]`.
To start you off you already have one evaluation at the mid-point.

In [134]:
blackbox1 = bb.GPBlackBox(ndim=1)
blackbox1.plot_xy().properties(width=800, height=300)

alt.Chart(...)

Now you should choose which input point (`x0`) to evaluate the function at:

In [135]:
x0 = widgets.FloatSlider(
    value=0,
    min=bb.DOMAIN_MIN,
    max=bb.DOMAIN_MAX,
    step=0.01,
    description='x0:',
    readout_format='.2f',
)
display(x0)

FloatSlider(value=0.0, description='x0:', max=1.0, min=-1.0, step=0.01)

Evaluate the function and plot all the noisy evaluations `y` so far:

In [142]:
y = blackbox1([x0.value])[0][0]
print(f'Evaluated black box at {x0.value}; result={y}')
blackbox1.plot_xy().properties(width=800, height=300)

Evaluated black box at -0.49; result=1.2666390588693373


alt.Chart(...)

If you have finished evaluating the function at different points and you are confident where the maximum is, you can make a guess before executing the cells below.

Now show the function as a line and the noisy data we received as evaluations of it:

In [143]:
f1 = blackbox1.sample_f(100)
chart1f = (
    alt.Chart(f1)
    .mark_line()
    .encode(x='x0', y='f'))
chart1y = blackbox1.plot_xy()
chart1 = alt.layer(chart1y, chart1f)
chart1.properties(width=800, height=300)

alt.LayerChart(...)

### Bayesian optimisation for 1-dimensional input
Now we will try Bayesian optimisation using the [Trieste](https://secondmind-labs.github.io/trieste/) package which uses [GPflow](https://gpflow.readthedocs.io/en/master/) for Gaussian process regression.
This section is based on the Trieste tutorial.

In [204]:
model1 = gpflow.models.GPR((f1['x0'].to_numpy().reshape((-1, 1)), f1['f'].to_numpy().reshape((-1, 1))),
                           kernel=gpflow.kernels.Matern52(lengthscales=.4),
                           noise_variance=1.1e-6)

def blackbox_bo1(x):
    print(f'Evaluating black-box: {x.shape}')
    mean, var = model1.predict_y(tf.reshape(x, (-1, 1)))
    return - bb.rng.normal(loc=mean, scale=np.sqrt(blackbox1.noise_variance))

In [205]:
from dataclasses import astuple

import gpflow
from gpflow.utilities import print_summary, set_trainable
import numpy as np
import tensorflow as tf

import trieste
from trieste.bayesian_optimizer import OptimizationResult
from trieste.utils.objectives import branin, mk_observer
from trieste.acquisition.rule import OBJECTIVE

#### Initial sample over search space

In [206]:
observer = mk_observer(blackbox_bo1, OBJECTIVE)
lower_bound = tf.cast([bb.DOMAIN_MIN], gpflow.default_float())
upper_bound = tf.cast([bb.DOMAIN_MAX], gpflow.default_float())
search_space = trieste.space.Box(lower_bound, upper_bound)

num_initial_points = 5
initial_query_points = search_space.sample(num_initial_points)
initial_data = observer(initial_query_points)

Evaluating black-box: (5, 1)


#### Model the objective function

In [207]:
kernel = gpflow.kernels.Matern52(lengthscales=0.4 * np.ones(1,))
gpr = gpflow.models.GPR(astuple(initial_data[OBJECTIVE]), kernel=kernel, noise_variance=blackbox1.noise_variance)
# set_trainable(gpr.likelihood, False)

model = {OBJECTIVE: trieste.models.create_model_interface(
    {
        "model": gpr,
        "optimizer": gpflow.optimizers.Scipy(),
        "optimizer_args": {"options": dict(maxiter=100)},
    }
)}

#### Optimisation loop

In [208]:
bo = trieste.bayesian_optimizer.BayesianOptimizer(observer, search_space)

result: OptimizationResult = bo.optimize(15, initial_data, model)

if result.error is not None:
    raise result.error

dataset = result.datasets[OBJECTIVE]

Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)
Evaluating black-box: (1, 1)


#### Explore results

In [209]:
query_points = dataset.query_points.numpy()
observations = dataset.observations.numpy()

arg_min_idx = tf.squeeze(tf.argmin(observations, axis=0))

print(f"query point: {query_points[arg_min_idx, :]}")
print(f"observation: {-observations[arg_min_idx, :]}")

query point: [-0.63594245]
observation: [2.47028443]


In [213]:
bo_f1 = model[OBJECTIVE].model.predict_f(f1['x0'].to_numpy().reshape((-1, 1)))
bo_f1.numpy()

AttributeError: 'tuple' object has no attribute 'numpy'

### 2-dimensional input
If you wish, you can try the same optimisation problem but for a function of two variables.

In [112]:
blackbox2 = bb.GPBlackBox(ndim=2)
x0 = widgets.FloatSlider(
    value=0,
    min=bb.DOMAIN_MIN,
    max=bb.DOMAIN_MAX,
    step=0.01,
    description='x0:',
    readout_format='.2f',
)
x1 = widgets.FloatSlider(
    value=0,
    min=bb.DOMAIN_MIN,
    max=bb.DOMAIN_MAX,
    step=0.01,
    description='x1:',
    readout_format='.2f',
)
w = widgets.Box([x0, x1])

Choose which input point (x0, x1) to evaluate the function at:

In [113]:
display(w)

Box(children=(FloatSlider(value=0.0, description='x0:', max=1.0, min=-1.0, step=0.01), FloatSlider(value=0.0, …

Evaluate the function and plot all the evaluations so far:

In [117]:
y = blackbox2([x0.value, x1.value])[0][0]
print(f'Evaluated black box at ({x0.value}, {x1.value}); result={y}')
blackbox2.plot_xy()

Evaluated black box at (0.51, -0.56); result=-0.8095590858357883


alt.Chart(...)

If you have finished evaluating the function at different points and you are confident where the maximum is, you can make a guess before executing the cells below.

Now show the underlying function f (without noise) as a heatmap and the noisy data we received as evaluations of it:

In [118]:
f2 = blackbox2.sample_f(51)
f2

,x0,x1,f
0,-1.00,-1.0,0.590030
1,-0.96,-1.0,0.694906
2,-0.92,-1.0,0.728644
3,-0.88,-1.0,0.666551
4,-0.84,-1.0,0.545399
...,...,...,...
2596,0.84,1.0,-0.287326
2597,0.88,1.0,-0.155572
2598,0.92,1.0,-0.017615
2599,0.96,1.0,0.111467


In [125]:
chart2f = (
    alt.Chart(f2)
    .mark_square(size=60)
    .encode(x=alt.X('x0:Q', scale=alt.Scale(domain=bb.DOMAIN)),
            y=alt.Y('x1:Q', scale=alt.Scale(domain=bb.DOMAIN)),
            color=alt.Color('f:Q', scale=alt.Scale(scheme=bb.COLOURSCHEME, domainMid=0))))
chart2y = blackbox2.plot_xy()
chart2 = alt.layer(chart2f, chart2y)
chart2.properties(width=800, height=300)

alt.LayerChart(...)

## Gaussian process model underlying our black boxes

In [ ]:
gpflow.utilities.print_summary(blackbox1.model)

In [ ]:
gpflow.utilities.print_summary(blackbox2.model)

## Bayesian optimisation

Use Bayesian optimisation to find a maximum.

In [ ]:
blackbox2.x.shape

In [ ]:
f2

In [ ]:
x2 = np.concatenate((blackbox2.x, np.array([f2['x0'], f2['x1']]).T))
x2

In [ ]:
y2 = np.array([blackbox2.y, np.zeros_like(blackbox2.y)]).T

In [ ]:
gpflow.models.VGP(data, kernel, likelihood)

How does this work in practice?
We create a black-box function that simulates Rudolph's measurements and use a standard Bayesian optimisation implementation.